##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Keras コンポーネントのチュートリアル

***TensorFlow Extended (TFX) の各コンポーネントの紹介***

注：この例は、Jupyter スタイルのノートブックで今すぐ実行できます。セットアップは必要ありません。「Google Colab で実行」をクリックするだけです

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/components_keras"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/components_keras.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/components_keras.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/tfx/components_keras.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

この Colab ベースのチュートリアルでは、TensorFlow Extended (TFX) のそれぞれの組み込みコンポーネントをインタラクティブに説明します。

ここではデータの取り込みからモデルのプッシュ、サービングまで、エンド ツー エンドの機械学習パイプラインのすべてのステップを見ていきます。

完了したら、このノートブックのコンテンツを TFX パイプライン ソース コードとして自動的にエクスポートできます。これは、Apache Airflow および Apache Beam とオーケストレーションできます。

注意: このノートブックは、TFX パイプラインでのネイティブ Keras モデルの使用を示しています。**TFX は TensorFlow 2 バージョンの Keras のみをサポートします**。

## 背景情報

このノートブックは、Jupyter/Colab 環境で TFX を使用する方法を示しています。 ここでは、インタラクティブなノートブックでシカゴのタクシーの例を見ていきます。

TFX パイプラインの構造に慣れるのには、インタラクティブなノートブックで作業するのが便利です。独自のパイプラインを軽量の開発環境として開発する場合にも役立ちますが、インタラクティブ ノートブックのオーケストレーションとメタデータ アーティファクトへのアクセス方法には違いがあるので注意してください。

### オーケストレーション

TFX の実稼働デプロイメントでは、Apache Airflow、Kubeflow Pipelines、Apache Beam などのオーケストレーターを使用して、TFX コンポーネントの事前定義済みパイプライン グラフをオーケストレーションします。インタラクティブなノートブックでは、ノートブック自体がオーケストレーターであり、ノートブック セルを実行するときにそれぞれの TFX コンポーネントを実行します。

### メタデータ

TFX の実稼働デプロイメントでは、ML Metadata（MLMD）API を介してメタデータにアクセスします。MLMD は、メタデータ プロパティを MySQL や SQLite などのデータベースに格納し、メタデータ ペイロードをファイル システムなどの永続ストアに保存します。インタラクティブなノートブックでは、プロパティとペイロードの両方が、Jupyter ノートブックまたは Colab サーバーの `/tmp` ディレクトリにあるエフェメラル SQLite データベースに保存されます。

## セットアップ

まず、必要なパッケージをインストールしてインポートし、パスを設定して、データをダウンロードします。

### Pip のアップグレード

ローカルで実行する場合にシステム Pipをアップグレードしないようにするには、Colab で実行していることを確認してください。もちろん、ローカルシステムは個別にアップグレードできます。

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### TFX をインストールする

**注：Google Colab では、パッケージが更新されるため、このセルを初めて実行するときに、ランタイムを再起動する必要があります（[ランタイム]&gt; [ランタイムの再起動...]）。**

In [ ]:
!pip install -U tfx

## ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行するときにランタイムを再起動する必要があります（[ランタイム]&gt; [ランタイムの再起動...]）。 これは、Colab がパッケージを読み込むために必要ですです。

### パッケージをインポートする

標準の TFX コンポーネント クラスを含む必要なパッケージをインポートします。

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

ライブラリのバージョンを確認します。

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

### パイプライン パスを設定

In [ ]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/taxi_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

### サンプルデータのダウンロード

TFX パイプラインで使用するサンプル データセットをダウンロードします。

使用しているデータセットは、シカゴ市がリリースした [タクシートリップデータセット](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)です。 このデータセットの列は次のとおりです。

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

このデータセットを使用して、タクシー乗車の`tips`を予測するモデルを構築します。

In [ ]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

CSV ファイルを見てみましょう。

In [ ]:
!head {_data_filepath}

*注：このWeb サイトは、シカゴ市の公式 Web サイト www.cityofchicago.org で公開されたデータを変更して使用するアプリケーションを提供します。シカゴ市は、この Web サイトで提供されるデータの内容、正確性、適時性、または完全性について一切の表明を行いません。この Web サイトで提供されるデータは、いつでも変更される可能性があります。かかる Web サイトで提供されるデータはユーザーの自己責任で利用されるものとします。*

### InteractiveContext を作成する

最後に、このノートブックで TFX コンポーネントをインタラクティブに実行できるようにする InteractiveContext を作成します。

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

## TFX コンポーネントをインタラクティブに実行する

次のセルでは、TFX コンポーネントを 1 つずつ作成し、それぞれを実行して、出力アーティファクトを視覚化します。

### ExampleGen

`ExampleGen` コンポーネントは通常、TFX パイプラインの先頭にあり、以下を実行します。

1. データをトレーニング セットと評価セットに分割します (デフォルトでは、2/3 トレーニング + 1/3 評価)。
2. データを `tf.Example` 形式に変換します。 (詳細は[こちら](https://www.tensorflow.org/tutorials/load_data/tfrecord))
3. 他のコンポーネントがアクセスできるように、データを `_tfx_root` ディレクトリにコピーします。

`ExampleGen` は、データソースへのパスを入力として受け取ります。 ここでは、これはダウンロードした CSV を含む `_data_root` パスです。

注意: このノートブックでは、コンポーネントを 1 つずつインスタンス化し、`InteractiveContext.run()` で実行しますが、実稼働環境では、すべてのコンポーネントを事前に `Pipeline`で指定して、オーケストレーターに渡します（[TFX パイプライン ガイドの構築](https://www.tensorflow.org/tfx/guide/build_tfx_pipeline)を参照してください）。

#### Enabling the Cache

When using the `InteractiveContext` in a notebook to develop a pipeline you can control when individual components will cache their outputs.  Set `enable_cache` to `True` when you want to reuse the previous output artifacts that the component generated.  Set `enable_cache` to `False` when you want to recompute the output artifacts for a component, if you are making changes to the code for example.

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

`ExampleGen`の出力アーティファクトを調べてみましょう。このコンポーネントは、トレーニングサンプルと評価サンプルの 2 つのアーティファクトを生成します。

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

また、最初の 3 つのトレーニングサンプルも見てみます。

In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

`ExampleGen`がデータの取り込みを完了したので、次のステップ、データ分析に進みます。

### StatisticsGen

`StatisticsGen`コンポーネントは、データ分析用のデータセットの統計を計算し、ダウンストリームのコンポーネントで使用します。これは、[TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) ライブラリを使用します。

`StatisticsGen`コンポーネントは、データ分析用のデータセットの統計を計算し、ダウンストリーム コンポーネントで使用します。

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

`StatisticsGen` の実行が完了すると、出力された統計を視覚化できます。 色々なプロットを試してみてください！

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

`SchemaGen` コンポーネントは、データ統計に基づいてスキーマを生成します。（スキーマは、データセット内の特徴の予想される境界、タイプ、プロパティを定義します。）また、[TensorFlow データ検証](https://www.tensorflow.org/tfx/data_validation/get_started)ライブラリも使用します。

注意: 生成されたスキーマはベストエフォートのもので、データの基本的なプロパティだけを推論しようとします。確認し、必要に応じて修正する必要があります。

`SchemaGen` は、`StatisticsGen` で生成した統計を入力として受け取り、デフォルトでトレーニング分割を参照します。

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

`SchemaGen` の実行が完了すると、生成されたスキーマをテーブルとして視覚化できます。

In [ ]:
context.show(schema_gen.outputs['schema'])

データセットのそれぞれの特徴は、スキーマ テーブルのプロパティの横に行として表示されます。スキーマは、ドメインとして示される、カテゴリ特徴が取るすべての値もキャプチャします。

スキーマの詳細については、[SchemaGen のドキュメント](https://www.tensorflow.org/tfx/guide/schemagen)をご覧ください。

### ExampleValidator

`ExampleValidator` コンポーネントは、スキーマで定義された期待に基づいて、データの異常を検出します。また、[TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) ライブラリも使用します。

`ExampleValidator` は、`Statistics Gen{/code 1} からの統計と <code data-md-type="codespan">SchemaGen` からのスキーマを入力として受け取ります。

In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

`ExampleValidator` の実行が完了すると、異常をテーブルとして視覚化できます。

In [ ]:
context.show(example_validator.outputs['anomalies'])

異常テーブルでは、異常がないことがわかります。これは、分析した最初のデータセットで、スキーマはこれに合わせて調整されているため、異常がないことが予想されます。このスキーマを確認する必要があります。予期されないものは、データに異常があることを意味します。確認されたスキーマを使用して将来のデータを保護できます。ここで生成された異常は、モデルのパフォーマンスをデバッグし、データが時間の経過とともにどのように変化するかを理解し、データ エラーを特定するために使用できます。

### 変換

`Transform`コンポーネントは、トレーニングとサービングの両方で特徴量エンジニアリングを実行します。これは、 [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) ライブラリを使用します。

`Transform`は、`ExampleGen`からのデータ、`SchemaGen`からのスキーマ、ユーザー定義の Transform コードを含むモジュールを入力として受け取ります。

以下のユーザー定義の Transform コードの例を見てみましょう（TensorFlow Transform API の概要については、[チュートリアルを参照してください](https://www.tensorflow.org/tfx/tutorials/transform/simple)）。まず、特徴量エンジニアリングのいくつかの定数を定義します。

注意: `%%writefile` セル マジックは、セルの内容をディスク上の`.py`ファイルとして保存します。これにより、`Transform` コンポーネントはコードをモジュールとして読み込むことができます。


In [ ]:
_taxi_constants_module_file = 'taxi_constants.py'

In [ ]:
%%writefile {_taxi_constants_module_file}

NUMERICAL_FEATURES = ['trip_miles', 'fare', 'trip_seconds']

BUCKET_FEATURES = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]
# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

CATEGORICAL_NUMERICAL_FEATURES = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

CATEGORICAL_STRING_FEATURES = [
    'payment_type',
    'company',
]

# Number of vocabulary terms used for encoding categorical features.
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized categorical are hashed.
OOV_SIZE = 10

# Keys
LABEL_KEY = 'tips'
FARE_KEY = 'fare'

def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

次に、生データを入力として受け取り、モデルがトレーニングできる変換された特徴量を返す {code 0}preprocessing _fn を記述します。

In [ ]:
_taxi_transform_module_file = 'taxi_transform.py'

In [ ]:
%%writefile {_taxi_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_NUMERICAL_FEATURES = taxi_constants.NUMERICAL_FEATURES
_BUCKET_FEATURES = taxi_constants.BUCKET_FEATURES
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_CATEGORICAL_NUMERICAL_FEATURES = taxi_constants.CATEGORICAL_NUMERICAL_FEATURES
_CATEGORICAL_STRING_FEATURES = taxi_constants.CATEGORICAL_STRING_FEATURES
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY


def _make_one_hot(x, key):
  """Make a one-hot tensor to encode categorical features.
  Args:
    X: A dense tensor
    key: A string key for the feature in the input
  Returns:
    A dense one-hot tensor as a float list
  """
  integerized = tft.compute_and_apply_vocabulary(x,
          top_k=_VOCAB_SIZE,
          num_oov_buckets=_OOV_SIZE,
          vocab_filename=key, name=key)
  depth = (
      tft.experimental.get_vocabulary_size_by_name(key) + _OOV_SIZE)
  one_hot_encoded = tf.one_hot(
      integerized,
      depth=tf.cast(depth, tf.int32),
      on_value=1.0,
      off_value=0.0)
  return tf.reshape(one_hot_encoded, [-1, depth])


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[taxi_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

  for key in _BUCKET_FEATURES:
    outputs[taxi_constants.t_name(key)] = tf.cast(tft.bucketize(
            _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT, name=key),
            dtype=tf.float32)

  for key in _CATEGORICAL_STRING_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(_fill_in_missing(inputs[key]), key)

  for key in _CATEGORICAL_NUMERICAL_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(tf.strings.strip(
        tf.strings.as_string(_fill_in_missing(inputs[key]))), key)

  # Was this passenger a big tipper?
  taxi_fare = _fill_in_missing(inputs[_FARE_KEY])
  tips = _fill_in_missing(inputs[_LABEL_KEY])
  outputs[_LABEL_KEY] = tf.where(
      tf.math.is_nan(taxi_fare),
      tf.cast(tf.zeros_like(taxi_fare), tf.int64),
      # Test if the tip was > 20% of the fare.
      tf.cast(
          tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.2))), tf.int64))

  return outputs

次に、この特徴量エンジニアリング コードを `Transform`コンポーネントに渡し、実行してデータを変換します。

In [ ]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_transform_module_file))
context.run(transform, enable_cache=True)

`Transform`の出力アーティファクトを調べてみましょう。このコンポーネントは、2 種類の出力を生成します。

- `transform_graph` は、前処理演算を実行できるグラフです (このグラフは、サービングモデルと評価モデルに含まれます)。
- `transformed_examples`は前処理されたトレーニングおよび評価データを表します。

In [ ]:
transform.outputs

`transform_graph` アーティファクトを見てみましょう。これは、3 つのサブディレクトリを含むディレクトリを指しています。

In [ ]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

`transformed_metadata` サブディレクトリには、前処理されたデータのスキーマが含まれています。`transform_fn`サブディレクトリには、実際の前処理グラフが含まれています。`metadata`サブディレクトリには、元のデータのスキーマが含まれています。

また、最初の 3 つの変換された例も見てみます。

In [ ]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

`Transform`コンポーネントがデータを特徴量に変換したら、次にモデルをトレーニングします。

### トレーナー

`Trainer`コンポーネントは、TensorFlow で定義したモデルをトレーニングします。デフォルトでは、Trainer は Estimator API をサポートします。Keras API を使用するには、トレーナーのコンストラクターで`custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)`をセットアップして [Generic Trainer](https://github.com/tensorflow/community/blob/master/rfcs/20200117-tfx-generic-trainer.md) を指定する必要があります。

`Trainer` は、`SchemaGen`からのスキーマ、`Transform`からの変換されたデータとグラフ、トレーニング パラメータ、およびユーザー定義されたモデル コードを含むモジュールを入力として受け取ります。

以下のユーザー定義モデル コードの例を見てみましょう（TensorFlow Keras API の概要については、[チュートリアルを参照してください](https://www.tensorflow.org/guide/keras)）。

In [ ]:
_taxi_trainer_module_file = 'taxi_trainer.py'

In [ ]:
%%writefile {_taxi_trainer_module_file}

from typing import Dict, List, Text

import os
import glob
from absl import logging

import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_LABEL_KEY = taxi_constants.LABEL_KEY

_BATCH_SIZE = 40


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      tf_transform_output.transformed_metadata.schema)

def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # TODO(b/154085620): Convert the predicted labels from the model using a
    # reverse-lookup (opposite of transform.py).
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    return transformed_features

  return transform_features_fn


def export_serving_model(tf_transform_output, model, output_dir):
  """Exports a keras model for serving.
  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    model: A keras model to export for serving.
    output_dir: A directory where the model will be exported to.
  """
  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          _get_transform_features_signature(model, tf_transform_output),
  }

  model.save(output_dir, save_format='tf', signatures=signatures)


def _build_keras_model(tf_transform_output: TFTransformOutput
                       ) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    tf_transform_output: [TFTransformOutput], the outputs from Transform

  Returns:
    A keras Model.
  """
  feature_spec = tf_transform_output.transformed_feature_spec().copy()
  feature_spec.pop(_LABEL_KEY)

  inputs = {}
  for key, spec in feature_spec.items():
    if isinstance(spec, tf.io.VarLenFeature):
      inputs[key] = tf.keras.layers.Input(
          shape=[None], name=key, dtype=spec.dtype, sparse=True)
    elif isinstance(spec, tf.io.FixedLenFeature):
      # TODO(b/208879020): Move into schema such that spec.shape is [1] and not
      # [] for scalars.
      inputs[key] = tf.keras.layers.Input(
          shape=spec.shape or [1], name=key, dtype=spec.dtype)
    else:
      raise ValueError('Spec type is not supported: ', key, spec)
  
  output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
  output = tf.keras.layers.Dense(100, activation='relu')(output)
  output = tf.keras.layers.Dense(70, activation='relu')(output)
  output = tf.keras.layers.Dense(50, activation='relu')(output)
  output = tf.keras.layers.Dense(20, activation='relu')(output)
  output = tf.keras.layers.Dense(1)(output)
  return tf.keras.Model(inputs=inputs, outputs=output)


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, _BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, _BATCH_SIZE)

  model = _build_keras_model(tf_transform_output)

  model.compile(
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  # Export the model.
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)

次に、このモデル コードを`Trainer`コンポーネントに渡し、それを実行してモデルをトレーニングします。

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_taxi_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer, enable_cache=True)

#### TensorBoard でトレーニングを分析する

トレーナーのアーティファクトを見てみましょう。これはモデルのサブディレクトリを含むディレクトリを指しています。

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

オプションで、TensorBoard を Trainer に接続して、モデルの学習曲線を分析できます。

In [ ]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

### Evaluator

`Evaluator` コンポーネントは、評価セットに対してモデル パフォーマンス指標を計算します。[TensorFlow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started)ライブラリを使用します。`Evaluator`は、オプションで、新しくトレーニングされたモデルが以前のモデルよりも優れていることを検証できます。これは、モデルを毎日自動的にトレーニングおよび検証する実稼働環境のパイプライン設定で役立ちます。このノートブックでは 1 つのモデルのみをトレーニングするため、`Evaluator`はモデルに自動的に「good」というラベルを付けます。

`Evaluator`は、`ExampleGen`からのデータ、`Trainer`からのトレーニング済みモデル、およびスライス構成を入力として受け取ります。スライス構成により、特徴値に関する指標をスライスすることができます (たとえば、午前 8 時から午後 8 時までのタクシー乗車でモデルがどのように動作するかなど)。 この構成の例は、以下を参照してください。

In [ ]:
# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=taxi_constants.LABEL_KEY,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(
            feature_keys=['trip_start_hour'])
    ])

次に、この構成を `Evaluator`に渡して実行します。

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=True)

evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=True)

`Evaluator` の出力アーティファクトを調べてみましょう。

In [ ]:
evaluator.outputs

`evaluation`出力を使用すると、評価セット全体のグローバル指標のデフォルトの視覚化を表示できます。

In [ ]:
context.show(evaluator.outputs['evaluation'])

スライスされた評価メトリクスの視覚化を表示するには、TensorFlow Model Analysis ライブラリを直接呼び出します。

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='trip_start_hour')

この視覚化は同じ指標を示していますが、評価セット全体ではなく、`trip_start_hour`のすべての特徴値で計算されています。

TensorFlow モデル分析は、公平性インジケーターやモデル パフォーマンスの時系列のプロットなど、他の多くの視覚化をサポートしています。 詳細については、[チュートリアル](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic)を参照してください。

構成にしきい値を追加したため、検証出力も利用できます。{code 0}blessing{/code 0} アーティファクトの存在は、モデルが検証に合格したことを示しています。これは実行される最初の検証であるため、候補は自動的に bless されます。

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

検証結果レコードを読み込み、成功を確認することもできます。

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

### Pusher

`Pusher` コンポーネントは通常、TFX パイプラインの最後にあります。このコンポーネントはモデルが検証に合格したかどうかをチェックし、合格した場合はモデルを `_serving_model_dir`にエクスポートします。

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

次に`Pusher`の出力アーティファクトを調べてみましょう。

In [ ]:
pusher.outputs

特に、Pusher はモデルを次のような SavedModel 形式でエクスポートします。

In [ ]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

組み込みの TFX コンポーネントの紹介は以上です。